In [77]:
import uproot
import awkward
import numpy as np

In [2]:
!which python

/global/homes/x/xju/.conda/envs/py3.9/bin/python


In [3]:
print(uproot.__version__)

5.0.6


You may have to update the syntax if you are using a different `uproot5` version.

In [4]:
filename = "/global/cfs/cdirs/m3443/data/TracklessBtagging/v1/Delphe_Hbb_5k_s1234.root"
tree_name = "Delphes"

In [5]:
tree = uproot.open(filename)[tree_name]

## Print all branches in the TTree

### Branches of interest

We are interested in:

* "Jet" (jet object, which we will identify b-jets from), 
* "Track" (trajectories associated with a jet), 
* and "Tower" (energy deposits associated with a jet)



In [7]:
tree.keys()

['Event',
 'Event/Event.fUniqueID',
 'Event/Event.fBits',
 'Event/Event.Number',
 'Event/Event.ReadTime',
 'Event/Event.ProcTime',
 'Event/Event.ProcessID',
 'Event/Event.MPI',
 'Event/Event.Weight',
 'Event/Event.CrossSection',
 'Event/Event.CrossSectionError',
 'Event/Event.Scale',
 'Event/Event.AlphaQED',
 'Event/Event.AlphaQCD',
 'Event/Event.ID1',
 'Event/Event.ID2',
 'Event/Event.X1',
 'Event/Event.X2',
 'Event/Event.ScalePDF',
 'Event/Event.PDF1',
 'Event/Event.PDF2',
 'Event_size',
 'Particle',
 'Particle/Particle.fUniqueID',
 'Particle/Particle.fBits',
 'Particle/Particle.PID',
 'Particle/Particle.Status',
 'Particle/Particle.IsPU',
 'Particle/Particle.M1',
 'Particle/Particle.M2',
 'Particle/Particle.D1',
 'Particle/Particle.D2',
 'Particle/Particle.Charge',
 'Particle/Particle.Mass',
 'Particle/Particle.E',
 'Particle/Particle.Px',
 'Particle/Particle.Py',
 'Particle/Particle.Pz',
 'Particle/Particle.P',
 'Particle/Particle.PT',
 'Particle/Particle.Eta',
 'Particle/Particle.

In [66]:
branches = ["Jet.PT", "Jet.Eta", "Jet.Phi", "Jet.Constituents", "Jet.Flavor", 
            "Tower.ET", "Tower.Eta", "Tower.Phi", "Tower.fUniqueID",
            "Track.PT", "Track.Eta", "Track.Phi", "Track.fUniqueID",
            "Particle.PT"
           ]

Loop over all events in the Root file. But only look at the first event!

In [67]:
for array in tree.iterate(branches, step_size=1):
    break

In [68]:
array

<Array [{'Jet.PT': [82.1, ...], ...}] type='1 * {"Jet.PT": var * float32, "...'>

### Jet properties

There are three jets in the event!

In [69]:
array["Jet.PT"], array["Jet.Eta"], array["Jet.Flavor"]

(<Array [[82.1, 55.3, 50]] type='1 * var * float32'>,
 <Array [[0.778, 1.51, -0.829]] type='1 * var * float32'>,
 <Array [[5, 5, 21]] type='1 * var * uint32'>)

Jet flavor of `5` means it is a b-jet (our signal); otherwise, it is a background jet.

### Tracks and Towers

In [115]:
track_ids = array["Track.fUniqueID"][0].to_numpy()
print(array["Track.fUniqueID"][0])
print("# of Tracks in the event:", len(array["Track.PT"][0]))

[915, 916, 917, 918, 919, 920, 921, 922, ..., 941, 942, 943, 944, 945, 946, 947]
# of Tracks in the event: 33


In [112]:
tower_ids = array["Tower.fUniqueID"][0].to_numpy()
tower_et = array["Tower.ET"][0].to_numpy()
print(tower_ids.shape, tower_et.shape)
print("# of Towers in the event:", len(array["Tower.ET"][0]))

(110,) (110,)
# of Towers in the event: 110


From these *tracks* and *towers*, we used an algorithm to reconstruct *three* jets in the event. So the jets should have connections to the tracks and towers.

### Particles

In [57]:
print("# of particles: ", len(array["Particle.PT"][0]))

# of particles:  685


In [13]:
array["Jet.Constituents"]

<Array [[{fName: '', fSize: 10, ...}, ...]] type='1 * var * TRefArray[fName...'>

In [22]:
len(array["Jet.Constituents"][0])

3

Let's look at the first jet's contituents.

In [74]:
contituent = array["Jet.Constituents"][0][0]

### Tracks and Towers associated with Jets

In [114]:
towers_in_jet = np.isin(tower_ids, contituent.refs.to_numpy())
tower_et_in_jet = tower_et[towers_in_jet]
tower_et_in_jet

array([ 1.9069964 ,  1.0189619 ,  0.620902  ,  0.90101975,  0.40306544,
        5.269566  , 17.173923  , 20.521887  , 22.022043  ,  9.112416  ],
      dtype=float32)

In [117]:
tracks_in_jet = np.isin(track_ids, contituent.refs.to_numpy())
np.sum(tracks_in_jet)

0

This means no tracks associated with jets!

Now use this as an example to do the visulization.